<a href="https://colab.research.google.com/github/XinzhouLi/Toxic_Language_Detection_in_Social_Media/blob/main/LSTM_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone https: // github.com/XinzhouLi/Toxic_Language_Detection_in_Social_Media.git


In [11]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils import data
from torch import nn, optim
import torchtext
import re


In [2]:
def unify_format(text):
    return re.sub(r"[^a-zA-Z0-9]", " ", text).lower().split()


In [3]:
# implement dataset extends util.data.Dataset
class ToxicDataset(data.Dataset):
    def __init__(self, *args):
        super().__init__()
        # Read data in
        if len(args) == 1:
            # for initialize training dataset
            filepath = args[0]
            self.dataframe = pd.read_csv(
                filepath, iterator=True, header=0, encoding='utf-8', usecols=['comment_text', 'toxic'])
            # initialize pandas dataframe to store the data
            self.dataframe = pd.DataFrame(self.dataframe.read())
            self.dataframe.convert_dtypes()
            # Clean the data using regular expersion, only reserve letter and number
            self.dataframe['comment_text'] = self.dataframe['comment_text'].apply(
                unify_format)
        else:
            # for initialize testing dataset
            filepath1 = args[0]
            filepath2 = args[1]
            test_comments = pd.DataFrame(pd.read_csv(
                filepath1, iterator=True, header=0, usecols=['comment_text', 'id']).read())
            test_label = pd.DataFrame(pd.read_csv(
                filepath2, iterator=True, header=0, usecols=['id', 'toxic']).read())
            # merge two dataframe together by unique key id
            result = pd.merge(test_comments, test_label, how='left', on=['id'])
            # select the text and label col
            self.dataframe = result[result['toxic']>= 0].loc[:, ['comment_text', 'toxic']]
            self.dataframe.convert_dtypes()
            # Clean the data using regular expersion, only reserve letter and number
            self.dataframe['comment_text'] = self.dataframe['comment_text'].apply(unify_format)

    # override the getiem function, return tuple contains comment and label

    def __getitem__(self, index):
        comment, label = self.dataframe.iat[index, 0], self.dataframe.iat[index, 1]
        return comment, label

    # Override the len function, return the length of the dataframe
    def __len__(self):
        return len(self.dataframe.loc[:, ['comment_text']])


In [4]:
# init dataset
train_dataset = ToxicDataset("train.csv")
test_dataset = ToxicDataset("test.csv", "test_labels.csv")
# init dataloader
train_dataloader = data.DataLoader(train_dataset, batch_size=100, shuffle=True)
test_dataloader = data.DataLoader(test_dataset, batch_size=100, shuffle=True)


In [18]:
# Testing code 
print(train_dataset.__getitem__(110))
print(torchtext.__version__)
text = torchtext.vocab.GloVe()
print(text.vectors.shape())

(['including', 'some', 'appropriate', 'mention', 'of', 'the', 'solomon', 'article', 'is', 'not', 'without', 'some', 'level', 'of', 'support'], 0)
0.14.1


RuntimeError: [enforce fail at C:\cb\pytorch_1000000000000\work\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 2635220400 bytes.

In [ ]:

class Network(nn.Module):
    def __init__(self, vocab_size, embeding_dim, hidden_dim):
        super().__init__()
        # implement embedding function to convert text to numerical data
        self.embedding = nn.Embedding(vocab_size, embeding_dim)
        self.rnn = nn.LSTM(embeding_dim, hidden_dim, num_layer =2, bidirectional=True, dropout = 0.5)
        self.fc = nn.Linear(hidden_dim*2,1)
        self.dropout = nn.Dropout(0.5)
    def forward(self, x):
        embediding = self.dropout(self.embedding(x))
        output,(hidden, cell) = self.rnn(embediding)
        hidden = torch.cat([hidden[-2], hidden[-1]],dim=1)
        hidden = self.dropout(hidden)
        out = self.fc(hidden)
        return out


In [20]:
vocab_size = 10000
embeding_size = 300


ValueError: iAt based indexing can only have integer indexers